In [ ]:
# Imports
import re
import nltk
from nltk.tokenize.moses import MosesDetokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.text import Text, TextCollection

In [ ]:
# Main idea behind generating stories and simplified stories is that you can do preprocessing on
# simplified_stories to help you choose sentences, but still include the original sentences in the
# summary (if you want).


# Read in data from target, breaking each story into sentences
def load_data_as_sentences(file, stem=False, remove_stop_words=False, lowercase=False):
    stories = []
    simplified_stories = []
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    # Load stories from file
    with open(file) as f:
        stories_raw = f.readlines()[:10] #TODO: After testing, remove [:10]
        
    for story in stories_raw:
        # Split story into a list of sentences
        story = re.sub("< newLine >", "\n", story)
        sentences = nltk.sent_tokenize(story)
        stories.append(sentences)
        
        # Extra preprocessing
        story_tokenized = [nltk.word_tokenize(sentence) for sentence in sentences]
        if remove_stop_words:
            story_tokenized = [[word for word in sentence if word not in stop_words] for sentence in story_tokenized]
        if stem:
            story_tokenized = [[stemmer.stem(word) for word in sentence] for sentence in story_tokenized]
        if lowercase:
            story_tokenized = [[word.lower() for word in sentence] for sentence in story_tokenized]
        
        simplified_stories.append(story_tokenized)
    return stories, simplified_stories

# Read in data from target, breaking each story into paragraphs (and then sentences)
def load_data_as_paragraphs(file, stem=False, remove_stop_words=False, lowercase=False):
    simplified_stories = []
    stories = []
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    # Load stories from file
    with open(file) as f:
        stories_raw = f.readlines()[:10] #TODO: After testing, remove [:10]
    for story in stories_raw:
        # Split into a list of paragraphs
        paragraphs = story.split("< newLine > < newLine >")
        simplified_paragraphs = []
        untokenized_paragraphs = []
        for paragraph in paragraphs:
            # SPlit paragraph into a list of sentences
            paragraph = re.sub("< newLine >", "\n", paragraph)
            sentences = nltk.sent_tokenize(paragraph)
            untokenized_paragraphs.append(sentences)
            # For the tokenized version, split each sentence int oa list of words
            paragraph_tokenized = [nltk.word_tokenize(sentence) for sentence in sentences]
            # Extra preprocessing
            if remove_stop_words:
                paragraph_tokenized = [[word for word in sentence if word not in stop_words] for sentence in paragraph_tokenized]
            if stem:
                paragraph_tokenized = [[stemmer.stem(word) for word in sentence] for sentence in paragraph_tokenized]
            if lowercase:
                paragraph_tokenized = [[word.lower() for word in sentence] for sentence in paragraph_tokenized]
                
            simplified_paragraphs.append(paragraph_tokenized)
        stories.append(untokenized_paragraphs)
        simplified_stories.append(simplified_paragraphs)
    return stories, simplified_stories



In [ ]:
# SumBasic algorithm


# Compute word probablities
def get_probs(data, tfidf=False, text=None):
    probs = {}
    if not tfidf:
        # Loop through each word
        for sentence in data:
            for word in sentence:
                # Build a dictionary of word frequency counts
                if word in probs:
                    probs[word] += 1
                else:
                    probs[word] = 1
        N = sum([len(sentence) for sentence in data]) * 1.0
        
        # Each word's score is word_count/total_word_count
        for key in probs.keys():
            probs[key] /= N
    else:
        # Otherwise each word's score is its tf_idf
        # NOTE: tf_idf was calculated over all stories, not just this one.
        for sentence in data:
            for word in sentence:
                probs[word] = all_tokens.tf_idf(word, text)
    return probs          

# Pick the best scoring sentence that contains the highest probability word.
def get_best_sentence(data, probs):
    highest_prob_word = max(probs, key=probs.get) 
    
    # Comment this out later, but for testing it's really useful to see what
    # words are being marked as most important
    print(highest_prob_word)
    best_sentence_index = -1
    best_score = 0.0
    for index, sentence in enumerate(data):
        if highest_prob_word in sentence:
            score = sum([probs[word] for word in sentence])/len(sentence)
            if score > best_score:
                best_score = score
                best_sentence_index = index
    return best_sentence_index

# Square the probablity of each word in the chosen sentence
def update_probs(probs, sentence):
    for word in set(sentence):
        probs[word] = probs[word] ** 2
    return probs


# Count sentences in text
def get_length(text):
    return len(text)

# Get tf_idf across all stories
def get_tfidf(data):
    text_list = [Text([word for sentence in story for word in sentence]) for story in data]
    story_tokens = TextCollection(text_list)
    return text_list, story_tokens

# Get tf_idf across all stories
def get_tfidf_paragraphs(data):
    text_list = [Text([word for paragraph in story for sentence in paragraph for word in sentence]) for story in data]
    story_tokens = TextCollection(text_list)
    return text_list, story_tokens

In [ ]:
# Actually run everything
stem = True
remove_stop_words = True
lowercase = True

# stories is a triply nested lists (first broken by story, then by paragraph, then by sentences)
# simplified_stories is a quadruply nested list (broken by story, paragraph, sentence, word)
stories, simplified_stories = load_data_as_paragraphs("writingPromptsData/train.wp_target", stem, remove_stop_words, lowercase)

#TODO: If necessary, introduce other cleaning things:
# - Clean up quotes in tetokenized story (or better yet, never tokenize at all)
# - Deal with parens unmatched
# - remove punctuation?


In [ ]:
summaries = []
detokenizer = MosesDetokenizer()
tfidf = True

texts, all_tokens = get_tfidf_paragraphs(simplified_stories)

# Loop through stories (assumes each story is a list of paragraphs, each of which are lists of sentences)
for story, simplified_story, text in zip(stories, simplified_stories, texts):
    summary = []
    # Loop through paragraphs, adding one sentence per paragraph to the summary.
    for index, paragraph, simplified_paragraph in zip(range(len(story)), story, simplified_story):
        # Get word probabilities
        probs = get_probs(simplified_paragraph, tfidf, text) 
        # Choose sentence with best score
        next_sentence_index = get_best_sentence(simplified_paragraph, probs)
        # Add it to summary
        summary.append(paragraph[next_sentence_index])
        # Update probabilities to downweight the words we just used.
        probs = update_probs(probs, simplified_paragraph[next_sentence_index])
    # Join sentences into a summary
    summary_string = " ".join(summary)
    print(summary_string)
    print(" ===== ")
    summaries.append(summary_string)


In [ ]:
max_length = 5

texts, all_tokens = get_tfidf(simplified_stories)
tfidf = False

# Note - this currently chooses sentences regardless of their order in the original text.  We could save sentence
# indexes instead, then use those to chop out the summary sentences in order.

# Original sumBasic (assumes stories are lists of sentences, not lists of paragraphs of sentences)
# to use this one, run the load_data_as_sentences function
for index, story, simplified_story, text in zip(range(len(stories)), stories, simplified_stories, texts):
    summary = []
    # Get word probabilities
    probs = get_probs(simplified_story, tfidf, text)
    # Keep adding sentences until we have enough
    while get_length(summary) <  max_length:
        # Choose sentence with best score
        next_sentence_index = get_best_sentence(simplified_story, probs)
        # Add it to summary
        summary.append(story[next_sentence_index])
        # Update probablities to downweight the words we just used
        probs = update_probs(probs, simplified_story[next_sentence_index])
    # Join sentences into a summary
    summary_string = " ".join(summary)
    print(summary_string)
    print(" ======= ")
    summaries.append(summary_string)